In [25]:
import pandas as pd

# Load the datasets
df_21_22 = pd.read_excel("Premier league 21_22.xlsx")
df_22_23 = pd.read_excel("Premier league 22_23.xlsx")
df_23_24 = pd.read_excel("Premier league 23_24.xlsx")
#Load the libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [26]:
# Step 1: Combine Datasets
df = pd.concat([df_21_22, df_22_23, df_23_24], ignore_index=True)

# Step 2: Selecting Columns of Interest
# Retaining meaningful columns, particularly goals, odds, and Asian handicap data
selected_columns = [
    'FTHG', 'FTAG', 'HTHG', 'HTAG',             # Goals scored (full-time and halftime)
    'B365H', 'B365D', 'B365A',                  # Bet365 odds for home win, draw, away win
    'IWH', 'IWD', 'IWA',                        # Alternative odds for win, draw, and loss
    'AHCh',                                     # Asian Handicap for home win
    'B365CAHH', 'B365CAHA',                     # Asian Handicap betting odds from Bet365
    'P>2.5', 'P<2.5',                           # Probability of over/under 2.5 goals
    'FTR'                                       # Target variable: Full-Time Result
]

# Keeping only these selected columns
df = df[selected_columns]

# Step 3: Handling Missing Values

# Dropping columns if missing values exceed a certain threshold (e.g., 30%)
missing_value_threshold = 0.3
for column in df.columns:
    if df[column].isnull().mean() > missing_value_threshold:
        df.drop(column, axis=1, inplace=True)

# Impute remaining missing values with the column mean for numerical columns
df.fillna(df.mean(numeric_only=True), inplace=True)

# Step 4: Verify Data Cleaning
# Check for any remaining missing values
print("Missing Values After Cleaning:\n", df.isnull().sum())

# Display the cleaned dataset
print("\nSample of Cleaned Data:\n", df.head())

Missing Values After Cleaning:
 FTHG        0
FTAG        0
HTHG        0
HTAG        0
B365H       0
B365D       0
B365A       0
IWH         0
IWD         0
IWA         0
AHCh        0
B365CAHH    0
B365CAHA    0
P>2.5       0
P<2.5       0
FTR         0
dtype: int64

Sample of Cleaned Data:
    FTHG  FTAG  HTHG  HTAG  B365H  B365D  B365A   IWH   IWD    IWA  AHCh  \
0     2     0     1     0   4.00   3.40   1.95  3.80  3.40   2.05  0.50   
1     5     1     1     0   1.53   4.50   5.75  1.55  4.40   5.75 -1.00   
2     1     2     1     0   3.10   3.10   2.45  3.15  3.05   2.45  0.25   
3     3     0     2     0   1.25   5.75  13.00  1.25  6.00  13.00 -1.50   
4     3     1     0     1   1.90   3.50   4.00  1.95  3.45   3.95 -0.50   

   B365CAHH  B365CAHA  P>2.5  P<2.5 FTR  
0      1.75      2.05   2.22   1.73   H  
1      2.05      1.75   1.67   2.32   H  
2      1.79      2.15   2.56   1.56   A  
3      2.05      1.75   1.80   2.09   H  
4      2.05      1.88   2.14   1.78   H  


In [27]:
# Selecting relevant features for predicting total goals
selected_features = [
    'FTHG', 'FTAG', 'HTHG', 'HTAG',  # Goals scored at full-time and halftime
    'B365H', 'B365D', 'B365A',       # Bet365 odds
    'AHCh',                          # Asian Handicap
    'B365CAHH', 'B365CAHA',          # Bet365 Asian Handicap odds
    'P>2.5', 'P<2.5'                 # Probabilities for over/under 2.5 goals
]

# Target variable: Total goals scored in a match
df['TotalGoals'] = df['FTHG'] + df['FTAG']  # Calculating total goals as target
target = df['TotalGoals']
features = df[selected_features]

# Handling missing values by filling with column mean
features.fillna(features.mean(), inplace=True)

<ipython-input-27-efc321646500>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.fillna(features.mean(), inplace=True)


In [28]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [29]:
# Model Selection - Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Model evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R^2 Score:", r2)

Mean Absolute Error: 0.017850877192982437
Mean Squared Error: 0.007631140350877191
R^2 Score: 0.9971342705233045
